In [ ]:
import os
import sys
import requests
import json
import openpyxl
from bs4 import BeautifulSoup
from openpyxl import Workbook
import pandas as pd
import numpy as np

headers = {
  'Host': 'openapi.naver.com',
  'User-Agent': 'curl/7.49..1',
  'Accept': '*/*',
  'X-Naver-Client-id': '0cRQYFHrJAQgcWi3njRi',
  'X-Naver-Client-Secret': 'vSOUBpiP02'
}

url = 'https://openapi.naver.com/v1/search/movie.json'

# 엑셀 파일 오픈
fileName = '/content/movie_list.xlsx'
myExcel = openpyxl.load_workbook(fileName,data_only=True)
# 시트 설정
mySheet = myExcel['korea']
get_cells = mySheet['A1388':'A4000']
idx = 1388

for row in get_cells:
  for cell in row:
    movie = cell.value
    years = mySheet.cell(row=idx, column=3).value
    #years = int(mySheet(idx+1, 3)).value
    #print(movie)
    params ={
        'query': movie,
        'yearfrom' : years,
        'yearto' : years
    }
    res = requests.get(url, headers=headers, params=params)
    #print(res.text)

    jData = json.loads(res.text)
    itemCnt=jData['total']
    
    for item in jData['items']:
      # 동명영화가 2개 이상 존재하는 경우
      if itemCnt > 1 :
        movName = item['title'].replace('<b>','')
        movName= movName.replace('</b>','')
        print('movieName='+movName)
        if mySheet['A'+str(idx)] == movName and \
        (item['director']==mySheet['F'+str(idx)] or (type(item['director'])=='list' and item['director'].split('|')[:-1].find(mySheet['F'+str(idx)])!=-1)) :
          # 배우 actor
          posActor= 'G'+ str(idx)
          actors= item['actor'].split('|')[:-1]
          mySheet[posActor] = ','.join(actors)
          # 평점 userRating
          mySheet['H'+str(idx)] = item['userRating']
          
      else:
        #actor
        posActor= 'G'+ str(idx)
        actors= item['actor'].split('|')[:-1]
        mySheet[posActor] = ','.join(actors)
        #score
        mySheet['H'+str(idx)] = item['userRating']
        
    print(res.text)
    
    idx=idx+1 
myExcel.save('/content/output.xlsx')

